In [0]:
dbutils.widgets.text("p_file_date", "2024-11-01")
v_file_date = dbutils.widgets.get("p_file_date")

Calculating average spending per customer

In [0]:
spark.sql("""
    CREATE TABLE IF NOT EXISTS clv_gold.average_customer_spending (
        customer_id STRING,
        avg_spend_per_visit DECIMAL(10, 2),
        created_date TIMESTAMP,
        updated_date TIMESTAMP
    )
    USING DELTA
""")


Out[73]: DataFrame[]

In [0]:
spark.sql(f"""
    CREATE OR REPLACE TEMP VIEW average_customer_spending_updated
    AS 
    SELECT 
        customer_id,
        ROUND(AVG(transaction_amount), 2) AS avg_spend_per_visit
    FROM 
        clv_silver.transaction_data
    WHERE 
        file_date = '{v_file_date}'
    GROUP BY        
        customer_id
""")

Out[84]: DataFrame[]

In [0]:
%sql
SELECT *
FROM average_customer_spending_updates

customer_id,avg_spend_per_visit
1,20.5
2,30.75


In [0]:
%sql
CREATE TABLE IF NOT EXISTS clv_gold.average_customer_spending
USING DELTA
AS 
SELECT 
    customer_id,
    ROUND (AVG(transaction_amount),2) AS Avg_Spend_Per_Visit
FROM 
    clv_silver.transaction_data
GROUP BY        
    customer_id

num_affected_rows,num_inserted_rows


In [0]:
%sql
MERGE INTO clv_gold.average_customer_spending AS target
USING average_customer_spending_updated AS source
ON target.customer_id = source.customer_id
WHEN MATCHED THEN
    UPDATE SET
        target.avg_spend_per_visit = source.avg_spend_per_visit,
        target.updated_date = current_timestamp
WHEN NOT MATCHED THEN
    INSERT (customer_id, avg_spend_per_visit,created_date)
    VALUES (source.customer_id, source.avg_spend_per_visit,current_timestamp)


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
30,30,0,0


In [0]:
%sql
SELECT *
FROM clv_gold.average_customer_spending

customer_id,avg_spend_per_visit,created_date,updated_date
CUST041,17.29,2024-11-13T21:57:54.781+0000,2024-11-13T22:01:33.704+0000
CUST021,17.47,2024-11-13T21:57:54.781+0000,2024-11-13T22:01:33.704+0000
CUST029,16.82,2024-11-13T21:57:54.781+0000,2024-11-13T22:01:33.704+0000
CUST024,16.72,2024-11-13T21:57:54.781+0000,2024-11-13T22:01:33.704+0000
CUST040,17.40,2024-11-13T21:57:54.781+0000,2024-11-13T22:01:33.704+0000
CUST026,19.71,2024-11-13T21:57:54.781+0000,2024-11-13T22:01:33.704+0000
CUST033,18.37,2024-11-13T21:57:54.781+0000,2024-11-13T22:01:33.704+0000
CUST050,19.53,2024-11-13T21:57:54.781+0000,2024-11-13T22:01:33.704+0000
CUST030,15.18,2024-11-13T21:57:54.781+0000,2024-11-13T22:01:33.704+0000
CUST037,16.33,2024-11-13T21:57:54.781+0000,2024-11-13T22:01:33.704+0000


Spending Pattern

a. Spending by Day of the Week:

In [0]:
"""
%sql
CREATE TABLE IF NOT EXISTS clv_gold.spending_per_day
USING DELTA
AS 
SELECT 
    DAYOFWEEK(transaction_date) AS Day_Of_Week,
    ROUND (AVG(transaction_amount),2) AS Avg_Spend_Per_Visit
FROM 
    clv_silver.transaction_data
GROUP BY 
    DAYOFWEEK(transaction_date)
ORDER BY 
    Day_Of_Week
  """

Out[79]: '\n%sql\nCREATE TABLE IF NOT EXISTS clv_gold.spending_per_day\nUSING DELTA\nAS \nSELECT \n    DAYOFWEEK(transaction_date) AS Day_Of_Week,\n    ROUND (AVG(transaction_amount),2) AS Avg_Spend_Per_Visit\nFROM \n    clv_silver.transaction_data\nGROUP BY \n    DAYOFWEEK(transaction_date)\nORDER BY \n    Day_Of_Week\n  '

Spending based on loyalty status

In [0]:

"""
%sql
CREATE TABLE IF NOT EXISTS clv_gold.spending_loyalty_status
USING DELTA
AS 
SELECT customer_info.loyal_status AS Loyalty, ROUND (AVG(transaction_amount),2) AS Avg_Spend
FROM clv_silver.transaction_data
JOIN clv_silver.customer_info WHERE customer_info.customer_id = clv_silver.transaction_data.customer_id
GROUP BY customer_info.loyal_status
"""

Out[80]: '\n%sql\nCREATE TABLE IF NOT EXISTS clv_gold.spending_loyalty_status\nUSING DELTA\nAS \nSELECT customer_info.loyal_status AS Loyalty, ROUND (AVG(transaction_amount),2) AS Avg_Spend\nFROM clv_silver.transaction_data\nJOIN clv_silver.customer_info WHERE customer_info.customer_id = clv_silver.transaction_data.customer_id\nGROUP BY customer_info.loyal_status\n'

Loyalty Program Impact

a. Loyalty Program Spending vs. Non-Loyalty

In [0]:
"""
%sql
CREATE TABLE IF NOT EXISTS clv_gold.loyalty_spending
USING DELTA
AS 
SELECT 
    c.loyal_status AS LoyaltyProgramStatus,
    ROUND (AVG(t.transaction_amount),2) AS Avg_Spend_Per_Visit,
    ROUND (AVG(v.visit_count),2) AS Avg_Visits_Per_Year
FROM 
    clv_silver.transaction_data AS t
JOIN 
    clv_silver.customer_info AS c ON t.customer_id = c.customer_id
JOIN 
    clv_silver.visit_frequency AS v ON t.customer_id = v.customer_id
GROUP BY 
    c.loyal_status
    """

Out[81]: '\n%sql\nCREATE TABLE IF NOT EXISTS clv_gold.loyalty_spending\nUSING DELTA\nAS \nSELECT \n    c.loyal_status AS LoyaltyProgramStatus,\n    ROUND (AVG(t.transaction_amount),2) AS Avg_Spend_Per_Visit,\n    ROUND (AVG(v.visit_count),2) AS Avg_Visits_Per_Year\nFROM \n    clv_silver.transaction_data AS t\nJOIN \n    clv_silver.customer_info AS c ON t.customer_id = c.customer_id\nJOIN \n    clv_silver.visit_frequency AS v ON t.customer_id = v.customer_id\nGROUP BY \n    c.loyal_status\n    '

b. Signup Date Analysis

In [0]:
"""
%sql
CREATE TABLE IF NOT EXISTS clv_gold.signup_date_analysis
USING DELTA
AS 
SELECT 
    t.customer_id,
    CASE 
        WHEN t.transaction_date< c.signup_date THEN 'Before Signup'
        ELSE 'After Signup'
    END AS Period,
    Round(AVG(t.transaction_amount),2) AS Avg_Spend_Per_Period
FROM 
    clv_silver.transaction_data AS t
JOIN 
    clv_silver.customer_info AS c ON t.customer_id = c.customer_id
GROUP BY 
    t.customer_id, Period
ORDER BY t.customer_id
"""

Out[82]: "\n%sql\nCREATE TABLE IF NOT EXISTS clv_gold.signup_date_analysis\nUSING DELTA\nAS \nSELECT \n    t.customer_id,\n    CASE \n        WHEN t.transaction_date< c.signup_date THEN 'Before Signup'\n        ELSE 'After Signup'\n    END AS Period,\n    Round(AVG(t.transaction_amount),2) AS Avg_Spend_Per_Period\nFROM \n    clv_silver.transaction_data AS t\nJOIN \n    clv_silver.customer_info AS c ON t.customer_id = c.customer_id\nGROUP BY \n    t.customer_id, Period\nORDER BY t.customer_id\n"

Item purchased


In [0]:
"""
%sql
CREATE TABLE IF NOT EXISTS clv_gold.item_purchased
USING DELTA
AS 
SELECT 
    item_purchased,
    COUNT(*) AS Purchase_Count
FROM 
    clv_silver.transaction_data
GROUP BY 
    item_purchased
ORDER BY 
    Purchase_Count DESC
"""

Out[83]: '\n%sql\nCREATE TABLE IF NOT EXISTS clv_gold.item_purchased\nUSING DELTA\nAS \nSELECT \n    item_purchased,\n    COUNT(*) AS Purchase_Count\nFROM \n    clv_silver.transaction_data\nGROUP BY \n    item_purchased\nORDER BY \n    Purchase_Count DESC\n'